# Client-Side Code

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image as kimage
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.models import Model

from PIL import Image

### Inference Model

In [28]:
# Half of the Inference Model Stored on Client Side
class ClientResNet(tf.keras.Model):
    def __init__(self, *args, **kwargs):
        super(ClientResNet, self).__init__(*args, **kwargs)
        
        # Load ResNet50 pre-trained model without top (fully connected) layers
        resnet_base = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
        resnet_base.trainable = False
        
        # Get the output of the first three convolutional layers
        middle_layer = resnet_base.get_layer('conv3_block4_out')
        # middle_layer = resnet_base.get_layer('conv5_block1_1_conv')
        self.seq0 = Model(inputs=resnet_base.input, outputs=middle_layer.output)

    # Convert image into right dimensions
    def preprocess_image(self, img):
        img_array = kimage.img_to_array(img)
        expand_img = np.expand_dims(img_array, axis=0)
        return preprocess_input(expand_img)

    # Forward pass of the client model
    def predict(self, image_path):
        img = Image.open(image_path)
        img = img.resize((224, 224)).convert("RGB")

        preprocessed_img = self.preprocess_image(img)
        return self.seq0(preprocessed_img)

In [29]:
# Load in Model
ClientModel = ClientResNet()

# Perform Inference on image path
image_path = "jeans.png"
client_embeddings = ClientModel.predict(image_path)

# Replace: This is a mimic for sending to server
np.save("client_embeddings.npy", client_embeddings)
client_embeddings.shape

TensorShape([1, 28, 28, 512])

In [30]:
image_path = "../polyvore_outfits/images/100004189.jpg"
client_embeddings = ClientModel.predict(image_path)
client_embeddings

<tf.Tensor: shape=(1, 28, 28, 512), dtype=float32, numpy=
array([[[[5.21752357e-01, 1.06648040e+00, 0.00000000e+00, ...,
          0.00000000e+00, 1.61465025e+00, 2.05138350e+00],
         [6.44631922e-01, 8.93821776e-01, 0.00000000e+00, ...,
          8.67959261e-01, 6.73466504e-01, 1.76751542e+00],
         [6.49372935e-01, 5.26428103e-01, 5.36130965e-02, ...,
          5.87304235e-01, 9.74427462e-01, 1.97073913e+00],
         ...,
         [6.87741339e-01, 2.93829679e-01, 0.00000000e+00, ...,
          3.59430462e-01, 1.02071166e+00, 2.03936648e+00],
         [3.75454664e-01, 5.88280976e-01, 0.00000000e+00, ...,
          0.00000000e+00, 1.24458539e+00, 1.75775480e+00],
         [5.12801707e-01, 1.31376755e+00, 0.00000000e+00, ...,
          0.00000000e+00, 1.60585284e+00, 2.05095220e+00]],

        [[2.77888149e-01, 2.61371851e-01, 0.00000000e+00, ...,
          9.37249511e-03, 9.38526034e-01, 1.69513607e+00],
         [5.43560863e-01, 2.24209800e-01, 0.00000000e+00, ...,
         

In [45]:
# Add Laplacian Noise
client_embeddings = ClientModel.predict(image_path)
noise_mean, noise_scale = 0.05, 0.1
laplace_noise = np.random.laplace(noise_mean, noise_scale, size=client_embeddings.shape)
client_embeddings += laplace_noise
np.save("laplace_embeddings.npy", client_embeddings)

In [46]:
# Add Siamese Noise

In [52]:
import tensorflow.keras.applications as keras_applications
model = keras_applications.ResNet50(weights='imagenet', include_top=False)
for layer in model.layers:
    print(layer.name, layer.output_shape)

input_37 [(None, None, None, 3)]
conv1_pad (None, None, None, 3)
conv1_conv (None, None, None, 64)
conv1_bn (None, None, None, 64)
conv1_relu (None, None, None, 64)
pool1_pad (None, None, None, 64)
pool1_pool (None, None, None, 64)
conv2_block1_1_conv (None, None, None, 64)
conv2_block1_1_bn (None, None, None, 64)
conv2_block1_1_relu (None, None, None, 64)
conv2_block1_2_conv (None, None, None, 64)
conv2_block1_2_bn (None, None, None, 64)
conv2_block1_2_relu (None, None, None, 64)
conv2_block1_0_conv (None, None, None, 256)
conv2_block1_3_conv (None, None, None, 256)
conv2_block1_0_bn (None, None, None, 256)
conv2_block1_3_bn (None, None, None, 256)
conv2_block1_add (None, None, None, 256)
conv2_block1_out (None, None, None, 256)
conv2_block2_1_conv (None, None, None, 64)
conv2_block2_1_bn (None, None, None, 64)
conv2_block2_1_relu (None, None, None, 64)
conv2_block2_2_conv (None, None, None, 64)
conv2_block2_2_bn (None, None, None, 64)
conv2_block2_2_relu (None, None, None, 64)
conv2_